# Simple Text Classifiers

This notebook will show a simple approach to text classification. Without any complicated pre-processing, linear and ensemble classification models will be tested. 

## Imports and Load Data

In [51]:
import numpy as np
import pandas as pd
from nltk import pos_tag
from nltk.corpus import wordnet, stopwords
from nltk.stem import snowball, WordNetLemmatizer
from nltk.tokenize import sent_tokenize, word_tokenize
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer 
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score
from sklearn.model_selection import train_test_split 
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import svm

In [2]:
file_name = "Isla Vista - All Excerpts - 1_2_2019.xlsx"
data = pd.read_excel(file_name, sheet_name='Dedoose Excerpts Export')
print(data.shape)
data = data.dropna(axis=0)
print(data.shape)
print(data.columns)

(8131, 53)
(8127, 53)
Index(['StoryID', 'Excerpt', 'CodesApplied_Combined', 'ACCOUNT',
       'ACCOUNT_Cultural', 'ACCOUNT_Individual', 'ACCOUNT_Other',
       'COMMUNITYRECOVERY', 'EVENT', 'GRIEF', 'GRIEF_Individual',
       'GRIEF_Community', 'GRIEF_Societal', 'HERO', 'INVESTIGATION', 'JOURNEY',
       'JOURNEY_Mental', 'JOURNEY_Physical', 'LEGAL', 'MEDIA', 'MISCELLANEOUS',
       'MOURNING', 'MOURNING_Individual', 'MOURNING_Community',
       'MOURNING_Societal', 'PERPETRATOR', 'PHOTO', 'POLICY', 'POLICY_Guns',
       'POLICY_InfoSharing', 'POLICY_MentalHealth', 'POLICY_Other',
       'POLICY_VictimAdv', 'POLICY_OtherAdv', 'POLICY_Practice',
       'PRIVATESECTOR', 'RACECULTURE', 'RESOURCES', 'SAFETY',
       'SAFETY_Community', 'SAFETY_Individual', 'SAFETY_SchoolOrg',
       'SAFETY_Societal', 'SOCIALSUPPORT', 'THREAT', 'THREAT_Assessment',
       'TRAUMA', 'TRAUMA_Physical', 'TRAUMA_Psychological',
       'TRAUMA_Individual', 'TRAUMA_Community', 'TRAUMA_Societal', 'VICTIMS'],
    

## Prepare Tokenizers

Two tokenizers will be tested, one with the most simple approach of stemming words. The second has some added complexity, using the WordNet lemmatizer.

In [3]:
excerpts = list(data['Excerpt'])
def stem_tokenizer(doc):
    stop_words = set(stopwords.words('english'))
    tokens = word_tokenize(doc) 
    stemmer = snowball.SnowballStemmer("english", ignore_stopwords=True)
    stemmed_tokens = [stemmer.stem(word) for word in tokens]
    list_tokens = [tok.lower() for tok in stemmed_tokens if tok.isalpha()]
    return(' '.join(list_tokens))
print("original: "+str(excerpts[3]))
print(stem_tokenizer(excerpts[3]))

original: A 22-year-old student last Friday killed six people and wounded 13 more in Isla Vista before turning his gun on himself. Commenters 
blamed the killer�s crimes on everything from misogynistic �pickup artist philosophy� to easy access to guns and no-fault divorce. Even 
�nerd culture� has come under scrutiny. 

Is American culture to blame for mass murder? 
a student last friday kill six peopl and wound more in isla vista before turn his gun on himself comment blame the crime on everyth from misogynist artist to easi access to gun and divorc even has come under scrutini is american cultur to blame for mass murder


In [4]:
excerpts = list(data['Excerpt'])
def lem_tokenizer(doc):
    stop_words = set(stopwords.words('english'))
    tokens = word_tokenize(doc) 
    lemmer = WordNetLemmatizer()
    lemmed_tokens = [lemmer.lemmatize(word) for word in tokens if word.lower() not in stop_words]
    list_tokens = [tok.lower() for tok in lemmed_tokens if tok.isalpha()]
    return(' '.join(list_tokens))
print("original: \n"+str(excerpts[3])+str("\n"))
print(lem_tokenizer(excerpts[3]))

original: 
A 22-year-old student last Friday killed six people and wounded 13 more in Isla Vista before turning his gun on himself. Commenters 
blamed the killer�s crimes on everything from misogynistic �pickup artist philosophy� to easy access to guns and no-fault divorce. Even 
�nerd culture� has come under scrutiny. 

Is American culture to blame for mass murder? 

student last friday killed six people wounded isla vista turning gun commenters blamed crime everything misogynistic artist easy access gun divorce even come scrutiny american culture blame mass murder


## Create Vectorizers

The two tokenizers can then be used to create vectorized representation. Two vectorizers will be used. First the count vectorizer, then the tfidf vectorizer.

In [5]:
# stem + count
docs = [stem_tokenizer(doc) for doc in excerpts]
vectorizer = CountVectorizer(max_features=1500, min_df=5, max_df=0.7, stop_words=stopwords.words('english'))  
stem_count_X = vectorizer.fit_transform(docs).toarray() 

In [6]:
# lem + count
docs = [lem_tokenizer(doc) for doc in excerpts]
vectorizer = CountVectorizer(max_features=1500, min_df=5, max_df=0.7, stop_words=stopwords.words('english'))  
lem_count_X = vectorizer.fit_transform(docs).toarray() 

In [7]:
# stem + tfidf
docs = [stem_tokenizer(doc) for doc in excerpts]
vectorizer = TfidfVectorizer(max_features=1500, min_df=5, max_df=0.7, stop_words=stopwords.words('english'))  
stem_tfidf_X = vectorizer.fit_transform(docs).toarray() 

In [8]:
# lem + tfidf
docs = [lem_tokenizer(doc) for doc in excerpts]
vectorizer = TfidfVectorizer(max_features=1500, min_df=5, max_df=0.7, stop_words=stopwords.words('english'))  
lem_tfidf_X = vectorizer.fit_transform(docs).toarray() 

##  Classifiers

Test each vectorized representation with simple classifiers.

### Linear

First compare two linear models: svm and logistic regression

#### SVM

In [9]:
docs_train, docs_test, y_train, y_test = train_test_split(stem_count_X, list(data['ACCOUNT']),
                                                          test_size=0.2, random_state=0) 
#Create a svm Classifier
clf = svm.SVC(kernel='linear') # Linear Kernel
#Train the model using the training sets
clf.fit(docs_train, y_train)

y_pred = clf.predict(docs_test)
print(confusion_matrix(y_test,y_pred))  
print(classification_report(y_test,y_pred))  
print(accuracy_score(y_test, y_pred))

[[1177   69]
 [  66  314]]
              precision    recall  f1-score   support

           0       0.95      0.94      0.95      1246
           1       0.82      0.83      0.82       380

    accuracy                           0.92      1626
   macro avg       0.88      0.89      0.88      1626
weighted avg       0.92      0.92      0.92      1626

0.9169741697416974


In [10]:
docs_train, docs_test, y_train, y_test = train_test_split(stem_tfidf_X, list(data['ACCOUNT']),
                                                          test_size=0.2, random_state=0) 
#Create a svm Classifier
clf = svm.SVC(kernel='linear') # Linear Kernel
#Train the model using the training sets
clf.fit(docs_train, y_train)

y_pred = clf.predict(docs_test)
print(confusion_matrix(y_test,y_pred))  
print(classification_report(y_test,y_pred))  
print(accuracy_score(y_test, y_pred))

[[1193   53]
 [  77  303]]
              precision    recall  f1-score   support

           0       0.94      0.96      0.95      1246
           1       0.85      0.80      0.82       380

    accuracy                           0.92      1626
   macro avg       0.90      0.88      0.89      1626
weighted avg       0.92      0.92      0.92      1626

0.9200492004920049


In [11]:
docs_train, docs_test, y_train, y_test = train_test_split(lem_count_X, list(data['ACCOUNT']),
                                                          test_size=0.2, random_state=0) 
#Create a svm Classifier
clf = svm.SVC(kernel='linear') # Linear Kernel
#Train the model using the training sets
clf.fit(docs_train, y_train)

y_pred = clf.predict(docs_test)
print(confusion_matrix(y_test,y_pred))  
print(classification_report(y_test,y_pred))  
print(accuracy_score(y_test, y_pred))

[[1173   73]
 [  73  307]]
              precision    recall  f1-score   support

           0       0.94      0.94      0.94      1246
           1       0.81      0.81      0.81       380

    accuracy                           0.91      1626
   macro avg       0.87      0.87      0.87      1626
weighted avg       0.91      0.91      0.91      1626

0.9102091020910209


In [12]:
docs_train, docs_test, y_train, y_test = train_test_split(lem_tfidf_X, list(data['ACCOUNT']),
                                                          test_size=0.2, random_state=0) 
#Create a svm Classifier
clf = svm.SVC(kernel='linear') # Linear Kernel
#Train the model using the training sets
clf.fit(docs_train, y_train)

y_pred = clf.predict(docs_test)
print(confusion_matrix(y_test,y_pred))  
print(classification_report(y_test,y_pred))  
print(accuracy_score(y_test, y_pred))

[[1188   58]
 [  75  305]]
              precision    recall  f1-score   support

           0       0.94      0.95      0.95      1246
           1       0.84      0.80      0.82       380

    accuracy                           0.92      1626
   macro avg       0.89      0.88      0.88      1626
weighted avg       0.92      0.92      0.92      1626

0.9182041820418204


#### Logistic Regression

Since all the setups gave equivalent f1-scores, the simples approach will be chosen to test with logisitic regression.

In [13]:
docs_train, docs_test, y_train, y_test = train_test_split(stem_count_X, list(data['ACCOUNT']), 
                                                          test_size=0.2, random_state=0) 

logreg = LogisticRegression(solver='lbfgs', max_iter=1000)
logreg.fit(docs_train, y_train)

y_pred = logreg.predict(docs_test)
print(confusion_matrix(y_test,y_pred))  
print(classification_report(y_test,y_pred))  
print(accuracy_score(y_test, y_pred))

[[1185   61]
 [  68  312]]
              precision    recall  f1-score   support

           0       0.95      0.95      0.95      1246
           1       0.84      0.82      0.83       380

    accuracy                           0.92      1626
   macro avg       0.89      0.89      0.89      1626
weighted avg       0.92      0.92      0.92      1626

0.9206642066420664


### Ensemble Model

Test the ensemble models random forest.

#### Count Vector

In [15]:
docs_train, docs_test, y_train, y_test = train_test_split(stem_count_X, list(data['ACCOUNT']), #test_size=0.2,
                                                         random_state=0) 
classifier = RandomForestClassifier(n_estimators=1000, random_state=0)  
classifier.fit(docs_train, y_train) 

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=1000,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [16]:
y_pred = classifier.predict(docs_test)  
print(confusion_matrix(y_test,y_pred))  
print(classification_report(y_test,y_pred))  
print(accuracy_score(y_test, y_pred))

[[1521   38]
 [ 110  363]]
              precision    recall  f1-score   support

           0       0.93      0.98      0.95      1559
           1       0.91      0.77      0.83       473

    accuracy                           0.93      2032
   macro avg       0.92      0.87      0.89      2032
weighted avg       0.93      0.93      0.92      2032

0.9271653543307087


In [17]:
print(len(classifier.feature_importances_) == len(vectorizer.get_feature_names()))
top_feats = np.argsort(classifier.feature_importances_)[-10:]
feat_names = [vectorizer.get_feature_names()[feat] for feat in top_feats]
print(feat_names)

True
['studying', 'requires', 'around', 'woodland', 'shifman', 'born', 'men', 'safe', 'month', 'village']


In [18]:
test_doc = np.zeros(len(docs_test[1]))
test_doc[top_feats[6]] = 1
print("class: "+str(classifier.predict([test_doc])))

class: [0]


The ensemble model shows the best performance, though it is important to note that the ensemble has a more complex decision boundary, and can be more prone to over-fitting. This would require evaluation through cross-validation to confirm the performance increase. It is also interesting to note the term "blame" was one of the top contributing terms to the model, and it results in the decision to classify a document as "account" (class label =1).

#### Tfidf Vector

In [35]:
# stem + tfidf
docs = [stem_tokenizer(doc) for doc in excerpts]
vectorizer = TfidfVectorizer(max_features=1500, min_df=5, max_df=0.7, stop_words=stopwords.words('english'))  
stem_tfidf_X = vectorizer.fit_transform(docs).toarray() 

In [36]:
docs_train, docs_test, y_train, y_test = train_test_split(pd.DataFrame(stem_tfidf_X), 
                                                          list(data['ACCOUNT']), #test_size=0.2,
                                                         random_state=0) 
classifier = RandomForestClassifier(n_estimators=1000, random_state=0)  
classifier.fit(docs_train, y_train) 

y_pred = classifier.predict(docs_test)  
print(confusion_matrix(y_test,y_pred))  
print(classification_report(y_test,y_pred))  
print(accuracy_score(y_test, y_pred))

[[1523   36]
 [ 102  371]]
              precision    recall  f1-score   support

           0       0.94      0.98      0.96      1559
           1       0.91      0.78      0.84       473

    accuracy                           0.93      2032
   macro avg       0.92      0.88      0.90      2032
weighted avg       0.93      0.93      0.93      2032

0.9320866141732284


In [20]:
print(len(classifier.feature_importances_) == len(vectorizer.get_feature_names()))
top_feats = np.argsort(classifier.feature_importances_)[-10:]
feat_names = [vectorizer.get_feature_names()[feat] for feat in top_feats]
print(feat_names)

True
['date', 'around', 'woodland', 'requires', 'shifman', 'born', 'safe', 'men', 'month', 'village']


In [59]:
print(confusion_matrix(y_test,y_pred))

[[1177   69]
 [  66  314]]


In [63]:
# documents incorrectly predicted with label 1
incorrect1_ids = [list(docs_test.index)[idx] for idx in range(0,len(docs_test)) 
              if (y_test[idx]==0 and y_pred[idx] == 1)]
incorrect1 = [docs[idx] for idx in incorrect1_ids]

# documents incorrectly predicted with label 0
incorrect0_ids = [list(docs_test.index)[idx] for idx in range(0,len(docs_test)) 
              if y_test[idx]==1 and y_pred[idx] == 0]
incorrect0 = [docs[idx] for idx in incorrect0_ids]

In [85]:
account_ratio = []
for i in range(50):
    top_features =100
    coef = classi.coef_.ravel()
    top_positive_coefficients = np.argsort(coef)[-top_features:]
    top_words = [list(count_vectorizer.get_feature_names())[idx] for idx in top_positive_coefficients]
    toks = incorrect1[i].split(" ")
    account_toks = [word for word in toks if word in top_words]
    account_ratio.append(len(account_toks)/(len(toks)))

In [86]:
example_ids = np.argsort(-np.array(account_ratio))
for example_id in example_ids[0:10]:
    toks = incorrect1[example_id].split(" ")
    print(str(toks)+" | "+str([word for word in toks if word in top_words]))
    #print([word for word in toks if word in top_words])

['polic', 'offic', 'who', 'spend', 'their', 'day', 'deal', 'with', 'violent', 'troubl', 'peopl', 'describ', 'rodger', 'as', 'polit', 'and', 'courteous'] | ['violent', 'troubl']
['elliot', 'rodger', 'bought', 'a', 'handgun', 'in', 'novemb', 'for', 'at', 'a', 'store', 'near', 'his', 'apart', 'he', 'later', 'bought', 'two', 'more'] | ['bought', 'bought']
['when', 'polic', 'encount', 'rodger', 'he', 'was', 'calm', 'he', 'was', 'ration', 'he', 'was', 'coher', 'and', 'without', 'other', 'evid', 'there', 'was', 'realli', 'noth', 'they', 'could', 'do', 'skinner', 'said'] | ['evid', 'skinner']
['deputi', 'describ', 'rodger', 'as', 'timid', 'and', 'in', 'their', 'interact', 'with', 'him', 'that', 'night'] | ['interact']
['we', 'have', 'a', 'session', 'about', 'safeti', 'and', 'risk', 'manag', 'with', 'everi', 'new', 'member', 'she', 'said', 'i', 'think', 'in', 'the', 'futur', 'this', 'will', 'be', 'someth', 'we', 'address'] | ['manag', 'address']
['a', 'santa', 'barbara', 'colleg', 'student', 'w

In [94]:
# documents incorrectly predicted with label 1
incorrect1_ids = [list(docs_test.index)[idx] for idx in range(0,len(docs_test)) 
              if (y_test[idx]==0 and y_pred[idx] == 1)]
incorrect1 = [docs[idx] for idx in incorrect1_ids]
top_words= ['screen','frustrat','america','notifi','fight','broken','none','dark',
            'irrespons','fail','date','count','welfar','role','footbal','brain',
            'hornaday','criteria','isol','blame']
account_ratio2 = []
incorrect1_toks = []
for i in range(50):
    toks = incorrect1[i].split(" ")
    account_toks = [word for word in toks if word in top_words]
    incorrect1_toks.append(account_toks)
    account_ratio2.append(len(account_toks)/(len(toks)))

In [96]:
# original docs
# documents incorrectly predicted with label 1
incorrect1_ids = [list(docs_test.index)[idx] for idx in range(0,len(docs_test)) 
              if (y_test[idx]==0 and y_pred[idx] == 1)]
incorrect1 = [excerpts[idx] for idx in incorrect1_ids]

example_ids = np.argsort(-np.array(account_ratio2))
for example_id in example_ids[0:3]:
    excerpt = incorrect1[example_id]
    acc_toks = incorrect1_toks[example_id]
    print(excerpt)
    print(acc_toks)
    print(str("****************"))

Rodger killed six students and wounded 13 other people last month near the University of California at Santa Barbara before taking his own life. He leftbehind videos and writings expressing sexual frustration and revealing his plans to kill women. 
['frustrat']
****************
A separate measure inspired by the Isla Vista tragedy, Senate Bill 505 by Sen. Hannah-Beth Jackson, D-Santa Barbara, was sent to
Brown earlier this week.
It would require law enforcement agencies to check the Department of Justice’s gun-ownership database whenever they are requested to
conduct a welfare check to determine whether the individual may be in possession of a firearm.
['welfar']
****************
Providing high school students with greater access to counselors, Bonamici said. Presumably, that would help address mental
health issues among youth. Practically speaking, focusing on the link between gun crimes and mental health may be the only area of
agreement among members of Congress, which has failed to

In [60]:
incorrect0[2]

'California has some of the strictest gun laws in the country. When Elliot Rodger\x92s frantic mother contacted authorities in April, Santa \nBarbara County sheriff\x92s deputies went to his apartment but found insufficient evidence to hold him. \n\nWhen, as a child, he displayed symptoms of emotional disturbance, his parents found therapists for him. The massacre happened \nanyway. Nonetheless, the inevitable question looms large as lawmakers, like the rest of us, search for sense in the aftermath of massacre. '

### Conclusions

These simple tests show that it should be possible to acheive a fairly high performance classifier, since these very basic pre-processing methods and simple linear classifier were able to achieve a fairly high f-score above 0.9. There is still room for improvement in the classification of the account class label, and one issue that can be seen is the class imbalance. It is not a major imbalance, but it will contribute to the lower f-score for class 1. 